$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$
$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$

This notebook will guide you in implementing QITE to obtain the ground state of $\hat{H} = \hat{Z}$. As stated in the question, we start from the initial state $\ket{+}$ and wish to propagate the state using a step size of $\Delta\tau = 0.1$ until $\beta = 2.5$. Let us begin by measuring the initial energy of this state.
<br>
<br>
1. Fill in the missing portion of the code below in the functions **ansatz** and **measure** (_2 marks_).

In [ ]:
import numpy as np
from pyquil.gates import *
from pyquil.noise import estimate_bitstring_probs
from pyquil import Program,get_qc

n = 25
shots = 5000
E = np.zeros([n+1],dtype=complex)
alist = []
def ansatz(p):
    # 1(a). Construct the ansatz for the initial state

def measure(p,ro,idx):
    # 1(b). We define measure as follow: We pass it an index idx that indicates which pauli expectation we want
    # idx = 0, measure "I"
    # idx = 1, measure "X"
    # idx = 2, measure "Y"
    # idx = 3, measure "Z"
    # I filled in the portion of the code for idx = 0, 1, your task is to fill in the portion for idx = 2 and 3.
    
    if idx == 0:
        return 1
    elif idx == 1:
        qc = get_qc('1q-qvm')
        p += H(0)
        p += MEASURE(0,ro[0])
        exe = qc.compile(p)
        res = qc.run(exe)
        probs = estimate_bitstring_probs(res)
        return probs[0] - probs[1]

    # 1(b) Fill in here
    elif idx == 2:
    
    # and here
    elif idx == 3:

def measure_energy(nn,alist,shots):
    p = Program()
    p.wrap_in_numshots_loop(shots)
    ro = p.declare('ro','BIT',1)
    ansatz(p)
    Energy = measure(p,ro,3)
    return Energy

E[0] = measure_energy(0,xlist,shots)
print('Initial energy is: ',E[0])

Now we try to implement the QITE protocol. It will be good to have a big picture of what should be done. We step through imaginary time and at each time step, we should first measure the expectation values of the pauli matrices $\sigma$. This is indicated in the first two lines of the for loop. Using the relevant expectation values, we obtain the coefficients $a[m]$ in equation (2.5) of Lecture 12 using the update rule in equation (2.9-2.11). We store it in a list and use this to propagate our state. Note we have to always reconstruct our state for each new measurements we make. Finally, we measure the current energy values. Let us construct the required functions starting with **get_expectation**

In [ ]:
'''
for i in range(1,n+1):
    sigma_expectation = np.zeros([4],dtype=complex)
    sigma_expectation = get_expectation(i,xlist,shots)
    update_alist(sigma_expectation,alist,db,shots)
    E[i] = measure_energy(i,alist,shots)
'''

$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$
$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$

We make use of the **measure** function we constructed earlier. The key here is to propagate our state using the coefficients in alist. We do this using the **propagate** function. In propagate **propagate**, we loop through the different values store in alist to construct the states. alist is stored as a list of lists and the indices are a[timestep][gate]. The indices of gate are as follows: 
<br>
<br>
1 stores a[x]: $e^{-ia[x]\Delta\tau\hat{X}}$
<br>
2 stores a[y]: $e^{-ia[y]\Delta\tau\hat{Y}}$
<br>
3 stores a[z]: $e^{-ia[z]\Delta\tau\hat{Z}}$
<br>
<br>
The 0 index stores the coefficient for the identity matrix $\hat{I}$, which is a global phase that we can ignore.
<br>
<br>
2. Fill in the missing portion of the code in the function **propagate**(_2 marks_).

In [ ]:
def get_expectation(i,alist,shots):
    sigma_expectation = np.zeros([4],dtype=complex)
    for j in range(4):
        p = Program()
        p.wrap_in_numshots_loop(shots)
        ro = p.declare('ro','BIT',1)
        ansatz(p)
        propagate(p,alist,i-1)
        sigma_expectation[j] = measure(p,ro,j)
    return sigma_expectation

def propagate(p,alist,i):
    if i == 0:
        None
    else:
        for t in range(i):
            for gate in range(1,4):
                angle = np.real(alist[t][gate])
                if gate == 1:
                    p += RX(angle,0)
                
                # 2(a) Fill in the missing gates here
                elif gate == 2:
                    # Here
                elif gate == 3:
                    # and here
                else:
                    raise ValueError

$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$
$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$

We now want to obtain the coefficients a[m] at the current time step and append to alist. To do this, we need to construct the matrix $S_{ij} = \bra{\psi}\sigma_{i}\sigma_{j}\ket{\psi}$ and the vector $b_{i}$. Well, our earlier functions allow us to measure the expectation values of the different pauli matrices. How do we obtain $S_{ij}$ from a list of $\bra{\psi}\sigma\ket{\psi}$? We can exploit the fact that up to some coefficients, $\sigma_{i}\sigma_{j} = c_{ij}\sigma_{ij}$. For example, $\sigma_{x}\sigma_{y} = i\sigma_{z}$. We will need a matrix to keep track of what pauli matrix and coefficient we get for $\sigma_{i}\sigma_{j}$. I constructed these and stored it in the matrices **index** and **coefficient** below.


In [ ]:
index = np.zeros([4,4],dtype=int)
index[0,0] = 0
index[0,1] = 1
index[0,2] = 2
index[0,3] = 3
index[1,0] = 1
index[1,1] = 0
index[1,2] = 3
index[1,3] = 2
index[2,0] = 2
index[2,1] = 3
index[2,2] = 0
index[2,3] = 1
index[3,0] = 3
index[3,1] = 2
index[3,2] = 1
index[3,3] = 0

coefficient = np.zeros([4,4],dtype=complex)
coefficient[0,0] = 1
coefficient[0,1] = 1
coefficient[0,2] = 1
coefficient[0,3] = 1
coefficient[1,0] = 1
coefficient[1,1] = 1
coefficient[1,2] = 1j
coefficient[1,3] = -1j
coefficient[2,0] = 1
coefficient[2,1] = -1j
coefficient[2,2] = 1
coefficient[2,3] = 1j
coefficient[3,0] = 1
coefficient[3,1] = 1j
coefficient[3,2] = -1j
coefficient[3,3] = 1

$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$
$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$

3. Using this, we can construct the S matrix and b vector. I did it for you in the code segment below. What I want is for you to use these, and the function np.linalg.std to solve for x in equation 2.9. Because the rotation gate is defined as $e^{-i\theta/2\sigma}$, you have to multiply 2 to the x you obtained before storing them in alist (_4 marks_).

In [ ]:
def update_alist(sigma_expectation,alist,db,shots):

    # Obtain S matrix
    S = np.zeros([4,4],dtype=complex)
    for i in range(4):
        for j in range(4):
            S[i,j] = sigma_expectation[index[i,j]]*coefficient[i,j]

    # Obtain b vector
    b = np.zeros([4],dtype=complex)
    c = 1
    wt = 1;
    hm = 3;
    c -= 2*db*wt*sigma_expectation[hm]
    c = np.sqrt(c)
    for i in range(4):
        b[i] += (sigma_expectation[i]/c-sigma_expectation[i])/(db)
        b[i] -= wt*coefficient[i,hm]*sigma_expectation[index[i,hm]]/c
        b[i] = -1j*b[i] + 1j*np.conj(b[i])

    # 3(a) Solve for x using np.linalg.lstsq and append your results to alist. Remember to multiply by 2.

$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$
$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$

4. The final thing to do is to modify **measure_energy** to make sure we measure the energy of the propagated states (_1 mark_).

In [ ]:
def measure_energy(nn,alist,shots):
    p = Program()
    p.wrap_in_numshots_loop(shots)
    ro = p.declare('ro','BIT',1)
    ansatz(p)
    
    # 4(a) Make sure we get the proper state before measuring the energy. Use propagate defined earlier.
    Energy = measure(p,ro,3)
    return Energy

5. Now plot how the energy varies with imaginary time (_1 mark_).

In [ ]:
db = 0.1
for i in range(1,n+1):
    sigma_expectation = np.zeros([4],dtype=complex)
    sigma_expectation = get_expectation(i,alist,shots)
    update_alist(sigma_expectation,alist,db,shots)
    E[i] = measure_energy(i,alist,shots)

# 5(a) Now plot the curve.